# Association Rule Mining in Mass-Spectrometry Data
### *Muhammad Haseeb and Muhammad Usman Tariq*
##### *CAP6778: Adv. Topics in Data Mining - Fall 2019*
##### *School of Computing and Information Sciences (SCIS)*
##### *Florida International University (FIU)*

### The MIT License

##### *Copyright 2019 Muhammad Haseeb and Muhammad Usman Tariq*

Permission is hereby granted, free of charge, to any person obtaining a copy of this software 
and associated documentation files (the "Software"), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial 
portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT 
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Required Imports

In [1]:
# Import the required libraries
import os
import re
import sys
import math
import time
import urllib
import operator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from IPython.display import clear_output

## Data Acquisition
Let's get the annotated HUMAN MS/MS spectral data libraries from <a href="https://chemdata.nist.gov/dokuwiki/doku.php?id=peptidew:cdownload" class="external">NIST.</a> Note that this can be done for any spectral library or multiple spectral libraries at once as well.

In [2]:
mouse = 'cptac2_mouse_hcd_selected.msp.tar.gz'
msp_file = './data/cptac2_mouse_hcd_selected.msp'
data_dir = os.path.expanduser("./data")

if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

if not os.path.exists(msp_file):
    if not os.path.exists(mouse):
        !wget 'ftp://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz'
    
    # Extract the tar ball into MSP file and move into the speclib folder
    !tar xvf 'cptac2_mouse_hcd_selected.msp.tar.gz'
    os.rename('./cptac2_mouse_hcd_selected.msp', msp_file)

### Data Extraction Hyperparameters

In [3]:
# Data extraction Hyperparamters
divider = 5                     # Divide every m/z by this number.
max_len = int(5000 / divider)  # Max allowed m/z value.
filter_peaks = 100              # How many peaks to keep after filtration.

In [4]:
'''To keep updating this dict for multiple files. Dont define it again.'''
pep_spec = {}

### MSP File Parser

Read the MSP file, extract the required ions and pre-prcocess spectra

In [5]:
f=open(msp_file, "r")
lines = f.readlines()
f.close()

isName = isMW = isNumPeaks = False
new = prev = 0
i = 0


while i < len(lines):
    '''
    Process each line of the file.
    '''
    line = lines[i]
    i += 1
    
    splits = line.split(':')
    
    '''Keep going through lines. Name line is peptide and
    all the lines after Num peaks are the spectrum'''
    if splits[0] == 'Name':
        split1 = splits[1]
        pep = split1.split('/')[0].lstrip(' ')
        isName = True
        
    if isName and splits[0] == 'MW':
        mass = float(splits[1]) # Not using it anywhere. It's just there.
        isMW = True
        
    if isName and isMW and splits[0] == 'Num peaks':
        '''After this line, we will start processing the spectrum.'''
        numPeaks = int(splits[1]) # Just in case.
        
        temp_spec = np.zeros(max_len)
        while (lines[i] != '\n'):
            mzline = lines[i]
            i += 1
            mzsplits = mzline.split('\t')
            # Get the m/z and intenisty value of each line.
            moz, intensity, pattern = float(mzsplits[0]), float(mzsplits[1]), mzsplits[2]

            # Filter out only the non-noisy terminal ions from the spectrum
            lbls = re.findall(r"(?<!\/)(?<![A-Z])([abcxyz][0-9]+)", pattern)

            if lbls:
                # Dividing m/z by the divider.
                temp_spec[round(moz / divider)] = intensity

        isNumPeaks = True
        
    if isName and isMW and isNumPeaks:
        '''At this point, we are done reading one spectrum.
        Place everything where it needs to go and read the next one.'''
        isName = isMW = isNumPeaks = False
        
        # This is the filter. Gets the top 100 peaks
        # and returns them in one hot encoded vector of size max_len
        top_indx = np.argpartition(temp_spec, -filter_peaks)[-filter_peaks:]
        spec = np.zeros(max_len, dtype=bool)
        spec[top_indx] = True
        
        # Creating dictionary. Key: peptide, Value: spectrum.
        # As a side effect we'll get rid of duplicates.
        if pep not in pep_spec: pep_spec[pep] = spec
        
        # Displays the progress.
        new = int((i/len(lines)) * 100)
        if new > prev:
            clear_output(wait=True)
            print(str(new) + '%')
            prev = new


99%


## Frequent Itemset Mining
Construct the market-basket dataframe.

### Frequent Itemset Hyperparameters

In [6]:
max_itemset_size = 6     # Max size of the frequent itemsets.
min_support = 0.45       # Maybe we should increase this?

### DataFrame Construction

In [7]:
# Just get the value from the dict.
buckets = list(pep_spec.values())
print(len(buckets))

# The apriori function needs a dataframe.
df_buckets = pd.DataFrame(buckets, columns=list(range(0, max_len)))

10026


In [8]:
df_buckets

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


### Apriori Algorithm
Using the association rule mining algorithms from: *Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994*. Using implementation from the [mlxtend.apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) Python package.

In [ ]:
freq_itemsets = apriori(df_buckets,
                        min_support=min_support, 
                        max_len=max_itemset_size, 
                        verbose=1,
                        low_memory=True)

Iteration: 17686048 | Sampling itemset size 6

### Freq Itemsets Visualization

Now that we have mined the frequent itemsets from the dataframe, let's visualize some stats and see if we need to refine any hyperparameters and mine itemsets again.

In [ ]:
# Add another column to store lengths of the frequent itemsets
freq_itemsets['length'] = freq_itemsets['itemsets'].apply(lambda x: len(x))

# Filter out the small itemsets (of length < 3)
filtered_freqITS = freq_itemsets[ (freq_itemsets['length'] > 2) &
                                        (freq_itemsets['support'] > 0.8) ]
# We found 625 filtered frequent itemsets
disp_ffITS = filtered_freqITS.reset_index().rename(columns={'index' : 'itemset#'})

filtered_freqITS = filtered_freqITS.drop(['length'], axis=1)

In [ ]:
disp_ffITS

We found about 756 frequent itemsets with support > 0.8 and length > 2. These look good. Let's mine the association rules.

## Association Rule Mining

Let's mine association rules from the obtained frequent itemsets and take a look at rules ($R = I\rightarrow j$) with high *confidence* ($Conf$) and *interest* ($Interest$):

$$
Conf(I\rightarrow j) = \frac{Support(I \cup j)}{Support(I)} \\
Interest(I \rightarrow j) = Conf(I\rightarrow j) - Pr [j]
$$
where $Pr [j]$ is the probability of the item $j$ computed as: $support(j)$

### Association Rules Hyperparameters

In [ ]:
confidence = 0.80     # Association Rule Confidence
intr   = 0.50     # Association Rule Interest (leverage)

### Association Rules
Using the association rule mining algorithms from: *Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994*. Using implementation from the [mlxtend.association_rules](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/) Python package.

In [ ]:
# Get association rules based on the confidence 
rules_conf = association_rules(freq_itemsets, 
                               metric='confidence', 
                               min_threshold=confidence, 
                               support_only=False)

In [ ]:
# Get association rules based on the interest
rules_conf['interest'] = rules_conf['confidence'] - rules_conf['consequent support']

### Association Rules Visualization

In [ ]:
rules_conf

# Results
So we have 750 rules of our interest (high *support, confidence* and *interest*). Let's dig deeper in these results.

In [ ]:
# Let's sort the results based on the leverage (interest)
res = rules_conf.sort_values(by=['interest'], ascending=False)
res = res[res['interest']>=intr]

In [ ]:
res

## Interpretation of Results

In [ ]:
df_buckets.columns = df_buckets.columns.map(str)
df_buckets.columns

In [ ]:
specs = np.array(df_buckets[(df_buckets['329'] == True) & 
                            (df_buckets['334'] == True) & 
                            (df_buckets['331'] == True) & 
                            (df_buckets['332'] == True) &
                            (df_buckets['333'] == True)].reset_index()['index'])

In [ ]:
seqs = list(pep_spec.keys())

In [ ]:
specs = np.array(df_buckets[(df_buckets['331'] == True) & 
           (df_buckets['363'] == True) & 
           (df_buckets['326'] == True) & 
           (df_buckets['328'] == True) &
           (df_buckets['249'] == True)].reset_index()['index'])

seqs = list(pep_spec.keys())

In [ ]:
# List of peptides having this common itemset

for kk in specs:
    print (seqs[kk])